In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

asignacion_path = 'bases/asignacion/base_asignacion_20240904_ TRABAJADA.xlsx'
df_asignacion = pd.read_excel(asignacion_path)
df_asignacion.shape

In [7]:
#df_exj = df_asignacion_test[df_asignacion_test['CARTERA'] == 'EXJ']

#gestores_permitidos = ['Richard Gonzales Chavez', 'Cynthia Rosales', 'Cynthia Llanos Rosales']
#df_bc_kstbc = df_asignacion_test[(df_asignacion_test['CARTERA'].isin(['BC', 'KSTBC'])) & (df_asignacion_test['GESTOR'].isin(gestores_permitidos))]

#df_filtrado = pd.concat([df_exj, df_bc_kstbc])
#df_filtrado.reset_index(drop=True, inplace=True)
#print(df_filtrado.shape)
#df_filtrado.head(5)

In [ ]:
df_asignacion_test = df_asignacion.copy()
print(df_asignacion_test.columns)

cols_required = ['codigo', 'cont_18', 'clave', 'nombre', 'Moneda', 'Capital', 'Camp', 'Gestor', 'CAMPAÑA', 'TIPO_CARTERA', 'AGENCIA CORRECTA', 'tipo_fondo', 'oficina', 'territorio', 'ID_VTA.']
df_asignacion_test = df_asignacion_test[cols_required]

df_asignacion_test.rename(columns={
    'codigo': 'CODIGO CENTRAL', 
    'cont_18': 'N° CONTRATO', 
    'clave': 'CARTERA', 
    'nombre': 'NOMBRE DEL CLIENTE', 
    'Moneda': 'MONEDA', 
    'Capital': 'DEUDA CAPITAL', 
    'Camp': '% CONDONACIÓN', 
    'Gestor': 'GESTOR', 
    'CAMPAÑA': 'MONTO A PAGAR', 
    'TIPO_CARTERA': 'TIPO CARTERA', 
    'AGENCIA CORRECTA': 'AGENCIA DE COBRANZA', 
    'tipo_fondo': 'TIPO DE FONDO', 
    'oficina': 'OFICINA', 
    'territorio': 'TERRITORIO', 
    'ID_VTA.': 'ID_VENTA'
}, inplace=True)

df_asignacion_test = df_asignacion_test[df_asignacion_test['TIPO CARTERA'] == 'UNSECURED']
df_asignacion_test = df_asignacion_test[df_asignacion_test['ID_VENTA'].isin(['-'])]
df_asignacion_test = df_asignacion_test[~df_asignacion_test['AGENCIA DE COBRANZA'].isin(['SIN AGENCIA'])]

df_asignacion_test['CODIGO CENTRAL'] = df_asignacion_test['CODIGO CENTRAL'].astype('Int64').astype(str).str.zfill(8)
df_asignacion_test['N° CONTRATO'] = df_asignacion_test['N° CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
df_asignacion_test['MONTO A CONDONAR'] = df_asignacion_test['DEUDA CAPITAL'] * df_asignacion_test['% CONDONACIÓN']
df_asignacion_test['TIPO DE FONDO'] = df_asignacion_test['TIPO DE FONDO'].fillna('NULL')
df_asignacion_test['MONTO A PAGAR'] = df_asignacion_test.apply(lambda x: "DEUDA TOTAL" if x['TIPO DE FONDO'] in ['REACTIVA', 'FAE', 'CRECER'] else x['MONTO A PAGAR'], axis=1)

correos = {
    ('ASESCOM RJ', 'BC'): 'karol.lopez@rjabogados.com',
    ('ASESCOM RJ', 'EXJ'): 'lilia.gonzales@rjabogados.com',
    ('ASESCOM RJ', 'KSTBC'): 'rosa.hoyos@rjabogados.com',
    ('CLASA MORA', 'BC'): 'jsalazar@clasaperu.com',
    ('CLASA MORA', 'EXJ'): 'jayra@clasaperu.com',
    ('CLASA MORA', 'KSTBC'): 'jsolis@clasaperu.com',
    ('MORNESE MORA', 'BC'): 'brenda.paz@mornese.pe, juan.mendoza@mornese.pe, informes@mornese.pe',
    ('MORNESE MORA', 'EXJ'): 'aparco@mornese.pe, juan.mendoza@mornese.pe, informes@mornese.pe',
    ('MORNESE MORA', 'KSTBC'): 'juan.mendoza@mornese.pe, geraldine.hinojosa@mornese.pe, informes@mornese.pe',
    ('Muñiz', None): 'szegarra@munizlaw.com',
    ('RodriguezAngobaldo', None): 'ysifuentes@er.com.pe'
}

telefonos = {
    ('ASESCOM RJ', 'BC'): '955013048, 932811608',
    ('ASESCOM RJ', 'EXJ'): '932812578, 932811752',
    ('ASESCOM RJ', 'KSTBC'): '979270273, 932810247, 945309798',
    ('CLASA MORA', 'BC'): '940125238, 977746384',
    ('CLASA MORA', 'EXJ'): '940619571, 977746384',
    ('CLASA MORA', 'KSTBC'): '936994661, 977746384',
    ('MORNESE MORA', 'BC'): '017057808, 953090160',
    ('MORNESE MORA', 'EXJ'): '01-7057893, 977288425',
    ('MORNESE MORA', 'KSTBC'): '017057807, 967140068, 994376233',
    ('Muñiz', None): '999868827',
    ('RodriguezAngobaldo', None): '924104114'
}

def asignar_correo(row):
    return correos.get((row['AGENCIA DE COBRANZA'], row['CARTERA']), correos.get((row['AGENCIA DE COBRANZA'], None), None))

def asignar_telefono(row):
    return telefonos.get((row['AGENCIA DE COBRANZA'], row['CARTERA']), telefonos.get((row['AGENCIA DE COBRANZA'], None), None))

df_asignacion_test['CORREO DE LA AGENCIA'] = df_asignacion_test.apply(asignar_correo, axis=1)
df_asignacion_test['TELÉFONO DE AGENCIA'] = df_asignacion_test.apply(asignar_telefono, axis=1)

df_asignacion_test['CONTACTO BBVA'] = 'recuperaciones@opplus.bbva.com'
df_asignacion_test['CORREO BBVAA'] = 'recuperaciones@opplus.bbva.com'
df_asignacion_test['TELÉFONO'] = '595-0030'

df_asignacion_test.sort_values(by=['CARTERA', 'CODIGO CENTRAL', '% CONDONACIÓN'], inplace=True)
df_asignacion_test.reset_index(drop=True, inplace=True)

cols_final = ['CODIGO CENTRAL', 'N° CONTRATO', 'CARTERA', 'NOMBRE DEL CLIENTE', 'MONEDA', 'DEUDA CAPITAL', '% CONDONACIÓN', 'MONTO A CONDONAR', 'MONTO A PAGAR', 'AGENCIA DE COBRANZA', 'TIPO DE FONDO', 'TELÉFONO DE AGENCIA', 'CORREO DE LA AGENCIA', 'CONTACTO BBVA', 'CORREO BBVAA' , 'TELÉFONO', 'OFICINA', 'TERRITORIO']
df_asignacion_test = df_asignacion_test[cols_final]

print(df_asignacion_test.shape)
df_asignacion_test.head(5)

In [11]:
df_asignacion_test.to_excel('bases/campañas/CAMPAÑAS SEPTIEMBRE 2024.xlsx', index=False)